In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
import gc
import os
import matplotlib.pyplot as plt
import seaborn as sns 
import lightgbm as lgb
import itertools
import pickle, gzip
import glob
from sklearn.preprocessing import StandardScaler
from tsfresh.feature_extraction import extract_features
np.warnings.filterwarnings('ignore')
import dask.dataframe as dd
import missingno as msno
from pandasql import sqldf
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.model_selection import KFold
import matplotlib.gridspec as gridspec
from sklearn import preprocessing
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

In [2]:
#Always seed the randomness of this universe
np.random.seed(51)

In [3]:
%%time
pred1 = dd.read_csv('test_pred_20.csv')
pred2 = dd.read_csv('test_pred_22.csv')
pred1 = pred1.compute()
pred2 = pred2.compute()
print(pred1.shape,pred2.shape)

(3492890, 16) (3492890, 16)
CPU times: user 33.1 s, sys: 3 s, total: 36.1 s
Wall time: 8.07 s


In [4]:
%%time
pred1 = pred1.reset_index(drop=True)
pred2 = pred2.reset_index(drop=True)

CPU times: user 166 ms, sys: 283 ms, total: 449 ms
Wall time: 447 ms


In [5]:
pred1['object_id'].equals(pred2['object_id'])

True

In [6]:
%%time
test_ensemble = pd.DataFrame()

CPU times: user 288 µs, sys: 36 µs, total: 324 µs
Wall time: 330 µs


In [7]:
coef_lgb = 0.7
coef_mlp = 0.3

In [8]:
%%time
test_ensemble = pred1.iloc[:,1:]*coef_lgb + pred2.iloc[:,1:]*coef_mlp

CPU times: user 314 ms, sys: 611 ms, total: 924 ms
Wall time: 920 ms


In [9]:
print(test_ensemble.shape)

(3492890, 15)


In [11]:
test_ensemble['object_id'] = pred1['object_id']
print(test_ensemble.shape)

(3492890, 16)


In [13]:
test_ensemble = test_ensemble[list(pred1.columns)]

In [14]:
test_ensemble.tail()

,object_id,class_6,class_15,class_16,class_42,class_52,class_53,class_62,class_64,class_65,class_67,class_88,class_90,class_92,class_95,class_99
3492885,130787966,0.000155,0.339116,0.000280,0.212325,0.217332,0.000304,0.073618,0.000110,0.001788,0.011834,0.003623,0.139035,0.000146,0.000332,0.245711
3492886,130787971,0.000221,0.014997,0.000235,0.065942,0.198715,0.000284,0.019161,0.315217,0.000486,0.031940,0.000916,0.348031,0.000223,0.003632,0.231252
3492887,130787974,0.000757,0.030948,0.000162,0.832959,0.015963,0.000433,0.100705,0.000050,0.000247,0.001288,0.000629,0.014109,0.000204,0.001545,0.114034
3492888,130788053,0.000059,0.760807,0.000083,0.121148,0.006421,0.000082,0.010168,0.066965,0.000266,0.000459,0.000122,0.032815,0.000078,0.000527,0.147601
3492889,130788054,0.000118,0.048495,0.000129,0.342181,0.366100,0.000237,0.039894,0.000171,0.000279,0.011054,0.001074,0.189757,0.000134,0.000376,0.227315


In [15]:
%%time
test_ensemble.to_csv('test_pred_25.csv',index=False)

CPU times: user 1min 32s, sys: 1.3 s, total: 1min 33s
Wall time: 1min 33s


In [16]:
test_ensemble['object_id'].equals(pred1['object_id'])

True

In [17]:
!kaggle competitions submit -c PLAsTiCC-2018 -f test_pred_25.csv -m "Message"

100%|█████████████████████████████████████| 1.06G/1.06G [2:38:19<00:00, 120kB/s]
Successfully submitted to PLAsTiCC Astronomical Classification